In [1]:
import numpy as np
import tensorflow as tf
from load_dataset_VGG16 import load_dataset
import matplotlib.pyplot as plt

In [2]:
x_train,t_train = load_dataset('./train_dataset_VGG16',convert_type='RGB',flatten=True,normalize=True,one_hot_label=True)
x_test,t_test = load_dataset('./test_dataset_VGG16',convert_type='RGB',flatten=True,normalize=True,one_hot_label=True)

Load label : Done!
Load img : Done!
Load label : Done!
Load img : Done!


In [3]:
def list_shuffle(datas,labels):
    ###データのシャッフル
    index_list = np.arange(0,datas.shape[0])
    np.random.shuffle(index_list)
    x_data = datas[index_list]
    t_data = labels[index_list]
    return x_data,t_data

In [4]:
x_train_shuffle,t_train_shuffle = list_shuffle(x_train,t_train)
x_test_shuffle,t_test_shuffle = list_shuffle(x_test,t_test)

In [5]:
X = tf.placeholder(tf.float32, shape=[None, 224*224*3],name="input")
t = tf.placeholder(tf.float32, shape=[None, 2])
keep_prob = tf.placeholder(tf.float32)
### 入力層
input_layer = tf.reshape(X, [-1,224,224,3])

In [6]:
### 畳込み層、プーリング層1
h_conv1 = tf.layers.conv2d(inputs=input_layer,filters=96,kernel_size=[11, 11],strides=(4,4),padding='same',activation=tf.nn.relu)
h_norm1 = tf.nn.local_response_normalization(h_conv1)
h_pool1 = tf.layers.max_pooling2d(inputs=h_norm1,pool_size=(3,3),strides=(2,2),padding='same')

### 畳込み層、プーリング層2
h_conv2 = tf.layers.conv2d(inputs=h_pool1,filters=256,kernel_size=[5, 5],padding='same',activation=tf.nn.relu)
h_norm2 = tf.nn.local_response_normalization(h_conv2)
h_pool2 = tf.layers.max_pooling2d(inputs=h_norm2,pool_size=(3,3),strides=(2,2),padding='same')

### 畳込み層、プーリング層3
h_conv3_1 = tf.layers.conv2d(inputs=h_pool2,filters=384,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
h_conv3_2 = tf.layers.conv2d(inputs=h_conv3_1,filters=384,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
h_conv3_3 = tf.layers.conv2d(inputs=h_conv3_2,filters=256,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)

h_pool3 = tf.layers.max_pooling2d(inputs=h_conv3_3,pool_size=(3,3),strides=(2,2),padding='same')

### 全結合層1

stddev = np.sqrt(1.0 / 7*7*256)
h_W_fc1 = tf.Variable(tf.truncated_normal([7*7*256,4096], stddev=stddev)) 
h_b_fc1 = tf.Variable(tf.constant(0.1, shape=[4096]))
h_pool3_flat = tf.reshape(h_pool3, [-1, 7*7*256])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, h_W_fc1) + h_b_fc1)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### 全結合層2

stddev = np.sqrt(1.0 / 4096)
h_W_fc2 = tf.Variable(tf.truncated_normal([4096,4096], stddev=stddev)) 
h_b_fc2 = tf.Variable(tf.constant(0.1, shape=[4096]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, h_W_fc2) + h_b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

### 出力層
stddev = np.sqrt(2.0 / 4096)
W_fc3 = tf.Variable(tf.truncated_normal([4096,2], stddev=stddev))
b_fc3 = tf.Variable(tf.constant(0.1, shape=[2]))
fc = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)
y_conv = tf.nn.softmax(fc,name="output")

### 損失
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=t, logits=y_conv))

### 学習op
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(loss)

### モデルの評価
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(t,1))
# 精度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
print(h_pool1)
print(h_pool2)
print(h_pool3)

Tensor("max_pooling2d/MaxPool:0", shape=(?, 28, 28, 96), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 14, 14, 256), dtype=float32)
Tensor("max_pooling2d_3/MaxPool:0", shape=(?, 7, 7, 256), dtype=float32)


In [ ]:
saver = tf.train.Saver()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 1

for i in range(5000):
    batch_mask = np.random.choice(x_train_shuffle.shape[0], batch_size)
    x_batch = x_train_shuffle[batch_mask]
    t_batch = t_train_shuffle[batch_mask]
    sess.run(train_step,feed_dict={X: x_batch, t: t_batch,keep_prob:0.5})
    if i % 100 == 0:
        train_acc, train_loss = sess.run([accuracy,loss], feed_dict={X: x_train_shuffle, t: t_train_shuffle,keep_prob:1.0})
        test_acc = sess.run(accuracy, feed_dict={X: x_test_shuffle, t: t_test_shuffle,keep_prob:1.0})
        print "[Train] step: %d, loss: %f, acc: %f, [Test] acc : %f" % (i, train_loss, train_acc,test_acc)
        
saver.save(sess, "./RoadSign-Alex-ckpt/roadsign-alex")